In [88]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from uszipcode import ZipcodeSearchEngine
from geopy.geocoders import Nominatim

import pandas as pd
import seaborn.apionly as sns
from datetime import date, datetime
from haversine import haversine

# statistics package
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from random import randint
# packages for mapping
from mpl_toolkits.basemap import Basemap

# packages for interactive graphs
from ipywidgets import widgets, interact
from IPython.display import display
from copy import deepcopy as copy
import time
from geopy.geocoders import Nominatim

from time import sleep
import requests
import time,datetime
%matplotlib inline

In [90]:
findzip = ZipcodeSearchEngine()

In [3]:
historical_data = pd.read_csv('clean_data.csv')

In [4]:
historical_data.head()

,Unnamed: 0,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,...,dropoff_hour,pickup_date,dropoff_date,distance,speed,day_interval,waiting_time,waiting_penalty,pickup_zipcode,dropoff_zipcode
0,186987,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 20:38:00,2013-01-13 20:58:00,1,1200,...,20,2013-01-13,2013-01-13,10.418191,31.254572,2,0.000000,0.000000,11371,10025
1,136205,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:08:00,2013-01-13 21:10:00,1,120,...,21,2013-01-13,2013-01-13,0.376856,11.305694,2,21.362104,0.029670,10023,10024
2,114412,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:31:00,2013-01-13 21:35:00,1,240,...,21,2013-01-13,2013-01-13,0.996570,14.948549,2,497.832494,0.691434,10016,10168
3,148407,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:39:00,2013-01-13 21:46:00,1,420,...,21,2013-01-13,2013-01-13,4.608048,39.497556,2,1872.055354,2.600077,10010,10278
4,114616,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 22:12:00,2013-01-13 22:15:00,1,180,...,22,2013-01-13,2013-01-13,1.231572,24.631431,2,3194.887668,4.437344,10003,10010


In [49]:
historical_data.keys()

Index(['Unnamed: 0', 'medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount', 'payment_type', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'pickup_day',
       'pickup_weekday', 'pickup_hour', 'pickup_minute', 'pickup_time',
       'dropoff_hour', 'pickup_date', 'dropoff_date', 'distance', 'speed',
       'day_interval', 'waiting_time', 'waiting_penalty', 'pickup_zipcode',
       'dropoff_zipcode'],
      dtype='object')

## Best Revenue from Historical Data

In [83]:
best_revenue = historical_data.groupby(['hack_license','pickup_date'])['total_amount'].sum()

In [87]:
np.max(best_revenue.reset_index()['total_amount'])

673.97

## Preprocess

In [89]:
historical_data["dropoff_minute"] = pd.to_datetime(historical_data["dropoff_datetime"]).apply(lambda x: x.minute)
historical_data["dropoff_time"] =historical_data["dropoff_hour"] + (historical_data["dropoff_minute"] / 60)

## Reinforcement Model

In [91]:
class TaxiWorld():
    def __init__(self):
        self.waiting_penalty = 5.00
        pass
                
    def getReward(self, row):
        if row['waiting_time']/3600<=2:
            return row['total_amount'] - row['waiting_time']/3600*self.waiting_penalty
        else:
            return row['total_amount']
        
    
    def set_waiting_penalty(self, penalty):
        self.waiting_penalty = penalty
        
    def getState(self):
        print('Current State')
        return self.s
    
    def setState(self, s):
        self.s = s
        
    def getStateSize(self, trip_and_fare):
        return len(trip_and_fare['zip_code'].unique())
    
    def getZipCodes(self, trip_and_fare):
        return self.zip_codes
    
    def getActionSize(self):
        return len(self.actions)

    def nextAction(self, s):
        print('Calculate Next Action based on state')
        
    def check_Q_table(self, s):
        if s not in self.Q:
            self.Q[s] = dict((action, 0.0) for action in self.actions)
            

In [92]:
env = TaxiWorld()

In [93]:
class TaxiRevenue:
    def __init__(self, env, penalty=5.0):
        self.env = env
        self.env.set_waiting_penalty(penalty)
        self.Q = dict()
        self.dropPickMap= dict()
        
    def generate_key(self, row):
        return str(row['pickup_zipcode']) + '_'  + str(row['pickup_weekday']) + '_' + str(row['day_interval'])
    
    def get_tuples(self, key):
        tuples = key.split('_')
        return {'pickup_zipcode': tuples[0],
                   'pickup_weekday': tuples[1],
                   'day_interval': tuples[2]}
    
    def get_meantriptime(self, data):
        map=historical_data.groupby(['pickup_zipcode', 'dropoff_zipcode'])['trip_time_in_secs'].mean()
        return map.reset_index()
    
    def isValidPickup(self, taxidata, dropPickMap):
        key=self.generate_key(taxidata)
        if key in dropPickMap:
            return True
        else:
            return False
        
    def generate_dropPickMap(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            if key not in self.dropPickMap:
                self.dropPickMap[key] = []
            if str(row['dropoff_zipcode']) not in self.dropPickMap[key]:
                
                self.dropPickMap[key].append(str(row['dropoff_zipcode']))
            
        return self.dropPickMap
    
    def getDropoff(self, key, dropPickMap):
        
        posDropoff= dropPickMap[key]
        print("dropoffs",posDropoff )
        rand=randint(0, len(posDropoff)-1)
        return posDropoff[rand]
         
    def getDayInterval(self,hour):
        if hour>=6 and hour<12:
            return 0
        elif hour>=12 and hour<18:
            return 1
        elif hour>=18 and hour<24:
            return 2
        else:
            return 3
        
    def getAction(self, taxidata, _Q,dropPickMap, const):
        best_reward=0
        #best_key = 0
        s = self.generate_key(taxidata)
        best_key=s
        #print(taxidata)
        if s in _Q:
            best_reward=_Q[s][0]*_Q[s][1]/const
            
            
        
        zipcode=findzip.by_zipcode(taxidata['dropoff_zipcode'])
        temp=taxidata
        possible_pickup=[]
        returns=100
        while not possible_pickup:
            pickup_res = findzip.by_coordinate(zipcode['Latitude'], zipcode['Longitude'], radius=10, returns=returns)
            for i in range(len(pickup_res)):
                temp['pickup_zipcode']=pickup_res[i]['Zipcode']
                #print(temp['pickup_zipcode'])
                if self.isValidPickup(temp, dropPickMap):
                    possible_pickup.append(pickup_res[i]['Zipcode'])

            print("valid pickup", possible_pickup)
            returns+=10
        
        for key in _Q:
            tuples=self.get_tuples(key)
               
            
            if tuples['pickup_zipcode'] in possible_pickup and tuples['pickup_weekday']==str(taxidata['pickup_weekday']) and tuples['day_interval']==str(taxidata['day_interval']):
            
                tempReward=_Q[key][0]*_Q[key][1]/const
                
                if tempReward>best_reward:
                    #print("in temp reward: ",tempReward)
                    #print("in best reward: ",best_reward)
                    best_key=key
                    best_reward= tempReward
                    #print(self.get_tuples(best_key))
                    
        return best_key
    
    def build_Q_table(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            reward = self.env.getReward(row)
            if key not in self.Q:
                self.Q[key] = [reward,1]
            else:
                self.Q[key] = [self.Q[key][0] +reward, self.Q[key][1] + 1]
                
    def greedy(self, s):
        return np.argmax(self.Q[s[0]]) 

    def epsilon_greed(self, epsilon, s):
        if np.random.rand() < epsilon:
            return np.random.randint(self.n_a)
        else:
            return self.greedy(s)
        
    def train(self, trip_and_fare):
        self.build_Q_table(trip_and_fare)
        return self.Q
        
    def distance(self, lat1, lon1, lat2, lon2):
        """calculates the Manhattan distance between 2 points
            using their coordinates

        Parameters
        ----------
        lat1: float
            latitude of first point

        lon1: float
            longitude of first point

        lat2: float
            latitude of second point

        lon2: float
            longitude of second point

        Returns
        -------
        d: float
            The Manhattan distance between the two points in kilometers

        """

        d = haversine((lat1, lon1), (lat2, lon1)) + haversine((lat2, lon1), (lat2, lon2))
        return d
    
    def find_distance(self, zip1, zip2):
        zipcode1=findzip.by_zipcode(zip1)
        zipcode2=findzip.by_zipcode(zip2)
        dis=self.distance(zipcode1['Latitude'], zipcode1['Longitude'],zipcode2['Latitude'], zipcode2['Longitude'])
        return dis
    
    
    def test(self, taxidata, _Q, dropPickMap,meanTrip, maxstep=1000):
        # TODO: Add your test routine that exmines learned policy
        #      Using the learned policy, generate the trajectory of 
        curr_dropoff_hour=taxidata['dropoff_time']
        #curr_pickup=taxidata['dropoff_time']
        end_time=curr_dropoff_hour+ 12
        
        trace = []
        rewards = []
        
        while curr_dropoff_hour <= end_time:
            
            best_key=self.getAction( taxidata, _Q,dropPickMap, 1000)
            pickup_tuple= self.get_tuples(best_key)
            
            if int(pickup_tuple['pickup_zipcode'])!=taxidata['dropoff_zipcode']:
                dis=self.find_distance(taxidata['dropoff_zipcode'], int(pickup_tuple['pickup_zipcode']))
                curr_dropoff_hour+= dis/taxidata['speed']
                print("roaming time", dis/taxidata['speed'] )
                
            dropoff=self.getDropoff(best_key, dropPickMap)
                
            print("going from ",pickup_tuple['pickup_zipcode'], " to ",  dropoff, " at ", curr_dropoff_hour)
            #curr_dropoff=curr_dropoff+meanTrip[meanTrip['pickup_zipcode']]
            #print(pickup_tuple['pickup_zipcode'])
            temptrip = meanTrip[(meanTrip['pickup_zipcode']==int(pickup_tuple['pickup_zipcode'])) & (meanTrip['dropoff_zipcode']==int(dropoff))]
            meanTripTime= temptrip.iloc[0]['trip_time_in_secs']
            #print(meanTripTime)
            
            curr_dropoff_hour=curr_dropoff_hour+meanTripTime/3600
            taxidata['dropoff_zipcode']=int(dropoff)
            taxidata['day_interval']=self.getDayInterval(curr_dropoff_hour)

## Train the Model

In [94]:
taxiRevenue = TaxiRevenue(env)

In [95]:
_Q = taxiRevenue.train(historical_data)

## Generate Test Data

In [ ]:
dropPickMap=taxiRevenue.generate_dropPickMap(historical_data)
meanTrip= taxiRevenue.get_meantriptime(historical_data)

## Test the Model

In [98]:
taxiRevenue.test(historical_data.iloc[14000], _Q, dropPickMap, meanTrip)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


valid pickup ['10036', '10018', '10001', '10020', '10019', '10199', '10119', '10112', '10111', '10173', '10165', '10069', '10168', '10167', '10170', '10154', '10016', '10174', '10022', '10011', '10023', '10017', '10010', '10065', '10014', '10003', '10021', '10044', '10024', '10075', '11109', '10009', '10028', '10012', '10128', '10013', '11101', '10025', '11222', '10278', '10282', '10002', '10007', '10029', '11106', '10103', '11102', '10026', '10006', '10038', '10280', '10115', '11104', '11211', '10027', '10004', '10035', '10005', '11201', '11103', '11206', '11105', '10037', '10030', '11377', '11205', '11378', '10031', '11370', '10451', '11237', '11217']
roaming time 0.09111998980611906
dropoffs ['10167', '10022', '10173', '10010', '10023', '10112', '10028', '10020', '10016', '10001', '10111', '10165', '10119', '10021', '10018', '10065', '10154', '10012', '10019', '10282', '10004', '10014', '10003', '10009', '10036', '10011', '10006', '10278', '10168', '10013', '11364', '11371', '10174'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



going from  10119  to  10112  at  7.391119989806119
valid pickup ['10112', '10111', '10020', '10173', '10154', '10167', '10165', '10170', '10022', '10168', '10174', '10017', '10036', '10018', '10016', '10019', '10119', '10065', '10199', '10001', '10023', '10044', '10069', '10021', '10010', '11109', '10075', '10011', '10028', '10024', '10003', '10009', '10014', '10128', '11101', '10012', '11222', '11106', '11102', '10025', '10029', '10013', '10002', '11104', '10278', '10026', '10103', '10282', '10007', '11103', '11211', '10035', '10115', '10038', '10006', '10027', '10280', '11105', '11377', '11201', '10004', '11206', '10005', '10037', '10030', '11378', '11370', '11205', '11237', '10031', '10451', '11372', '10039', '11217', '10455', '11373']
roaming time 0.1712826957785346
dropoffs ['10167', '10022', '10173', '10010', '10023', '10112', '10028', '10020', '10016', '10001', '10111', '10165', '10119', '10021', '10018', '10065', '10154', '10012', '10019', '10282', '10004', '10014', '10003', 

valid pickup ['10278', '10007', '10013', '10038', '10282', '10006', '10012', '10103', '10280', '10004', '10002', '10005', '10003', '11201', '10014', '10009', '10010', '10011', '10119', '11211', '10016', '10199', '11231', '11217', '10001', '10018', '11205', '10168', '10165', '11222', '10174', '10170', '10173', '10017', '10167', '10036', '11109', '11238', '10112', '11206', '10111', '10020', '10154', '11215', '10022', '11216', '10019', '11101', '10044', '10065', '11232', '10069', '11225', '11237', '10023', '10021', '11213', '10075', '11218', '11104', '11378', '10028', '11106', '10024', '11220', '11226', '10128', '11377', '11102', '11212', '11103', '10029', '10025']
roaming time 0.3865936219288146
dropoffs ['10167', '10022', '10173', '10010', '10023', '10112', '10028', '10020', '10016', '10001', '10111', '10165', '10119', '10021', '10018', '10065', '10154', '10012', '10019', '10282', '10004', '10014', '10003', '10009', '10036', '10011', '10006', '10278', '10168', '10013', '11364', '11371',